# Estrategias para vencer el bloqueo al hacer web scraping

1. **Detección de Chromedriver**: hay servicios que puede identificar si un navegador está controlado por herramientas de automatización como Chromedriver al analizar ciertos comportamientos y propiedades del navegador.

2. **Huellas Digitales del Dispositivo**: Si se utiliza la misma huella digital de navegador para un gran número de visitas, puede identificarse como comportamiento de máquina. Es necesario utilizar diferentes huellas digitales de navegador efectivas para distribuir las visitas.

3. **Detección de Proxies IP**: Bloqueo de ubicaciones IP maliciosas y limitación de la frecuencia de las solicitudes.

4. **Autenticidad del Navegador**: Verificar si los atributos del navegador y la información de la solicitud son anormales, como si el User-Agent en el encabezado está emitido por código Python y si el navegador declarado por el User-Agent tiene atributos correspondientes.

5. **Desafío JavaScript**: Enviar código JavaScript al usuario. Por lo general, los rastreadores no tienen la capacidad de renderizar directamente el JS. 

6. **Cookies**: Al verificar el período de validez de cf_clearance, se actualiza y rastrea continuamente si el comportamiento del usuario es anormal.

7. **Huella Digital TLS**: Los navegadores generalmente usan HTTP/2, pero las solicitudes realizadas por lenguajes de programación suelen predeterminarse a HTTP/1.1.

In [ ]:
import sys

In [ ]:
!{sys.executable} -m pip install undetected_chromedriver selenium_stealth

In [ ]:
import undetected_chromedriver as uc
from selenium_stealth import stealth

import time
from pathlib import Path

from bs4 import BeautifulSoup as bs

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# Usuarios MacOS
OS_PLATFORM = "mac-x64"

# Usuarios Windows
# OS_PLATFORM = "win64"

DRIVER = str(Path.home() / f"chromedriver-{OS_PLATFORM}/chromedriver")
BROWSER = r"/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
# BROWSER = r"C:\Program Files\Google\Chrome\Application\chrome.exe"

In [ ]:
def get_website(url, browser, headless=True, proxy_url=None, wait_=20):
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.140 Safari/537.36"
    options = uc.ChromeOptions()
    options.binary_location = browser
    if headless:
        options.add_argument('--headless=new')
    options.add_argument("--start-maximized")
    if proxy_url:
        options.add_argument(f"--proxy-server={proxy_url}")
    options.add_argument("user-agent={}".format(user_agent))
    driver = uc.Chrome(options=options)

    stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
            )

    driver.get(url)
    WebDriverWait(driver, wait_).until(
        lambda d: d.execute_script('return document.readyState') == 'complete')
    
    return driver

In [ ]:
url = 'https://www.plataformadetransparencia.org.mx'
website = get_website(url=url,
                      browser=BROWSER, 
                      headless=False)

¿Qué pasa si escaneamos la página en esta página web https://radar.cloudflare.com/scan?

Usemos BeautifulSoup para imprimir el contenido de la página en un formato "bonito"

In [ ]:
soup = bs(website.page_source, "html.parser")
print(soup.prettify())

## Localizadores de Selenium:

Son identificadores que se utilizan para encontrar elementos específicos en una página web. Loas más comunes son:

* Nombre de Clase: Seleccionar elementos con el atributo class. Ejemplo: `driver.find_element(By.CLASS_NAME, "información")`

* Selector CSS: Utiliza selectores CSS para localizar elementos. Ejemplo (para el elemento con id "nombre"): `driver.find_element(By.CSS_SELECTOR, "#nombre")`

* ID: Utiliza el atributo id único de un elemento. Ejemplo: `driver.find_element(By.ID, "apellido")`

* Nombre:  Identifica elementos con el atributo name. Ejemplo (para la casilla de verificación "boletín"): `driver.find_element(By.NAME, "boletin")`

* Texto del Enlace: Localiza enlaces basados en el texto que se muestra. Ejemplo: `driver.find_element(By.LINK_TEXT, "Página Oficial de Selenium")`

* Texto Parcial del Enlace: Selecciona enlaces usando una parte de su texto. Ejemplo: `driver.find_element(By.PARTIAL_LINK_TEXT, "Página Oficial")`

* Nombre de la Etiqueta: Utiliza la propia etiqueta HTML como localizador. Ejemplo (para la etiqueta "a" de un enlace): `driver.find_element(By.TAG_NAME, "a")`

* XPath: Emplea expresiones XPath para navegar por la estructura del documento HTML y localizar elementos. Ejemplo (para el botón de option F): `driver.find_element(By.XPATH, "//input[@value='f']")`

## ¿Debo usar selectores XPATH o CSS? 

**R.- DEPENDE:**

El *XPATH*, también conocido como XML PATH, es un lenguaje de consulta para esquemas XML. Permite localizar partes específicas dentro de un documento XML. Las expresiones XPAT se mueven a través del documento, desde el nodo raíz hasta cualquier elemento deseado en una página web.

**Sintaxis Básica de XPath**

| Expresión | Descripción |
|---|---|
| `node_name` | Selecciona todos los nodos con este nombre. |
| `/` | Selecciona desde el nodo raíz. |
| `.` | Selecciona el nodo actual. |
| `//` | Selecciona nodos desde el nodo actual que coinciden con la selección. |
| `@` | Selecciona atributos. |
| `..` | Selecciona el padre del nodo actual. |
| `*` | Selecciona cualquier nodo elemento. |
| `*@` | Selecciona cualquier atributo. |
| `node()` | Selecciona cualquier nodo. |

Por otra parte, un *selector CSS* es una representación de cadena de diferentes etiquetas HTML, clases, atributos e IDs. Algunas ventajas son su sintaxis simple y es más fácil de aprender que XPATH, lo malo es que si el DOM cambia, los selectores dejarán de funcionar.

* XPATH absoluto (**/**): `/html/body/div[2]/div[1]/div/h4[1]/b/html[1]/body[1]/div[2]/div[1]/div[1]/h4[1]/b[1]`
* Expresión CSS absoluta: `html.body.div.p.div.a`

El problema es ue si algo cambia, la expresión se romperá.

Para esos casos es mejor usar posiciones relativas

* Expresión XPATH relativa (**//**): `//tagname[@attribute=‘value’]`
* Expresión CSS relativa: `tagname[attribute=‘value’]`

In [ ]:
xpath_card = "/html/body/app-root/app-skeleton/div/div/div/app-home/div[3]/div/div/div/div[1]/div"
card_element = website.find_element(By.XPATH, xpath_card)
card_element.click()

Después de varios intentos, puede que Cloudfare bloquee tu IP. Podemos probar utilizando un proxy, un sistema o enrutador que actúa como intermediario proporcionando una nueva IP de acceso: https://www.proxynova.com/proxy-server-list/country-mx/

In [ ]:
PROXY_URL = "189.240.60.166:9090"
website = get_website(url=url,
                      browser=BROWSER,
                      proxy_url=PROXY_URL,
                      headless=False)

In [ ]:
xpath_card = "/html/body/app-root/app-skeleton/div/div/div/app-home/div[3]/div/div/div/div[1]/div"
card_element = website.find_element(By.XPATH, xpath_card)
card_element.click()

Puedes ser bloqueado de igual manera 😢

No se les olvide cerrar el driver.

In [ ]:
website.close()

En algunos intentos pude tener éxito agregando un tiempo de espera al elemento 'clickeable'

In [ ]:
website = get_website(url=url,
                      browser=BROWSER,
                      headless=False)

In [ ]:
clickable_element = WebDriverWait(website, 20).until(
    EC.element_to_be_clickable(By.XPATH, xpath_card)
)
clickable_element.click()

In [ ]:
Puedes ser bloqueado de igual manera X2 😢

A continuación les dejo algunos elementos para poder seleccionar con `website.find_element(By.XPATH, MIELEMENTO)` donde podemos reemplazar MIELEMENTO por los siguientes valores.

In [ ]:
entidad = '//*[@id="formEntidadFederativa:selectEntidad"]/option[7]'

In [ ]:
clickable_element = WebDriverWait(website, 20).until(
    EC.element_to_be_clickable(By.XPATH, entidad)
)
clickable_element.click()

In [ ]:
options_dict = {
    33: "Federación",
    1: "Aguascalientes",
    2: "Baja California",
    3: "Baja California Sur",
    4: "Campeche",
    5: "Coahuila de Zaragoza",
    6: "Colima",
    7: "Chiapas",
    8: "Chihuahua",
    9: "Ciudad de México",
    10: "Durango",
    11: "Guanajuato",
    12: "Guerrero",
    13: "Hidalgo",
    14: "Jalisco",
    15: "México",
    16: "Michoacán de Ocampo",
    17: "Morelos",
    18: "Nayarit",
    19: "Nuevo León",
    20: "Oaxaca",
    21: "Puebla",
    22: "Querétaro",
    23: "Quintana Roo",
    24: "San Luis Potosí",
    25: "Sinaloa",
    26: "Sonora",
    27: "Tabasco",
    28: "Tamaulipas",
    29: "Tlaxcala",
    30: "Veracruz",
    31: "Yucatán",
    32: "Zacatecas"
}

In [ ]:
sujeto_obligado = //*[@id="formEntidadFederativa:cboSujetoObligado"]

In [ ]:
clickable_element = WebDriverWait(website, 20).until(
    EC.element_to_be_clickable(By.XPATH, sujeto_obligado)
)
clickable_element.click()

In [ ]:
ejercicio = //*[@id="formEntidadFederativa:areaPeriodo"]

In [ ]:
tipo_obligacion = //*[@id="formEntidadFederativa:areaSelectTipoObligacion"]

In [ ]:
obligacion_general = //*[@id="formEntidadFederativa:areaSelectTipoObligacion"]/div[2]/div/span[1]
obligacion_especifica = //*[@id="formEntidadFederativa:areaSelectTipoObligacion"]/div[2]/div/span[2]

In [ ]:
todas_obligaciones = "//*[@id='formEntidadFederativa:j_idt856']"

obligaciones = {
    'todas_obligaciones': "//*[@id='myNavbar']/ul/li[2]",
    'determinacion_autoridad': "//*[@id='myNavbar']/ul/li[3]",
    'informes': "//*[@id='myNavbar']/ul/li[4]",
    'estadisticas': "//*[@id='myNavbar']/ul/li[5]",
    'atencion_ciudadania': "//*[@id='myNavbar']/ul/li[6]",
    'indicadores': "//*[@id='myNavbar']/ul/li[7]",
    'organizaion_interna': "//*[@id='myNavbar']/ul/li[8]",
    'uso_recursos_publicos': "//*[@id='myNavbar']/ul/li[9]"
}